In [13]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    FunctionTransformer
)
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV
#import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split


In [14]:
#load our data
data = pd.read_csv('data_2016.csv')


In [15]:
#lets train test split
X = data.drop(columns=['bought_highbrow_wines'])
y = data['bought_highbrow_wines']
#lets drop x values whose y is nan
X = X.loc[y.dropna().index]
y = y.dropna()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
def final_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    feature set based on behavioral relevance:
    - Intent signals (wine/premium purchase history)
    - Ability signals (spend capacity)
    - Willingness signals (price sensitivity, discount behavior)
    - Channel readiness (online purchase patterns)
    - Stable context (household, loyalty)
    """
    df = df.copy()
    
    # =========================================================================
    # STEP 1: Identify all category columns for aggregations
    # =========================================================================
    cat_cols = [c for c in df.columns if c.startswith("cat_")]
    
    # Coerce all cat_* to numeric
    for c in cat_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)
    
    # =========================================================================
    # STEP 2: Calculate TOTAL_SPEND (sum of all category purchases)
    # =========================================================================
    df["total_spend"] = df[cat_cols].sum(axis=1)
    
    # =========================================================================
    # STEP 3: ONLINE CHANNEL FEATURES
    # =========================================================================
    # Clean online metrics
    df["n_cogo"] = pd.to_numeric(df.get("n_cogo", 0), errors="coerce").fillna(0)
    df["cogo_rev"] = pd.to_numeric(df.get("cogo_rev", 0), errors="coerce").fillna(0)
    
    # Online ratio: what fraction of spend is online?
    df["online_ratio"] = np.where(
        df["total_spend"] > 0,
        df["cogo_rev"] / df["total_spend"],
        0
    )
    
    # =========================================================================
    # STEP 4: DISCOUNT BEHAVIOR
    # =========================================================================
    df["total_discount"] = pd.to_numeric(df.get("total_discount", 0), errors="coerce").fillna(0)
    
    # Discount ratio: how deal-dependent is this customer?
    df["discount_ratio"] = np.where(
        df["total_spend"] > 0,
        df["total_discount"] / df["total_spend"],
        0
    )
    
    # =========================================================================
    # STEP 5: WINE AFFINITY FEATURES
    # =========================================================================
    # Wine-adjacent premium foods that signal taste alignment
    wine_affinity_cols = [
    "cat_Wijn_Stillewijnen_RAYON",     # Still wines (anchor)
    "cat_Tapas",                      # Wine-paired appetizers
    "cat_KaasSeizoenskazen",           # Seasonal / specialty cheeses
    "cat_VerseKaasFruitkazen",         # Fresh / fruit cheeses
    "cat_VisGerookt",                  # Smoked fish
    "cat_VisVerseSchelpdieren",        # Fresh fish & shellfish
]

    wine_affinity_cols = [c for c in wine_affinity_cols if c in df.columns]
    
    df["wine_affinity_spend"] = df[wine_affinity_cols].sum(axis=1) if wine_affinity_cols else 0
    
    # Wine affinity ratio: lifestyle vs transactional
    df["wine_affinity_ratio"] = np.where(
        df["total_spend"] > 0,
        df["wine_affinity_spend"] / df["total_spend"],
        0
    )
    
    # =========================================================================
    # STEP 6: PREMIUM VS NECESSITY RATIO
    # =========================================================================
    # Premium lifestyle categories (discretionary, taste-driven)
    premium_cols = [
    "cat_Wijn_Stillewijnen_RAYON",   # Premium anchor
    "cat_Tapas",                    # Gourmet food
    "cat_KaasSeizoenskazen",         # Specialty cheese
    "cat_VisGerookt",               # Premium fish
    "cat_Bier_Genietbieren",         # Craft / premium beers
    "cat_Bloemen",                  # Gifting / discretionary
    "cat_ParfumerieEHBO",            # Personal care / premium
    "cat_Textiel_Bedlinnen",         # Lifestyle / home comfort
]
    premium_cols = [c for c in premium_cols if c in df.columns]
    
    # Necessity categories (survival shopping, family logistics)
    necessity_cols = [
        "cat_Babyluiers",                # Baby diapers
        "cat_Incontinentie_luiers",      # Adult diapers
        "cat_MelkKarnemelk",             # Basic dairy
        "cat_BroodKorthoudbaar",         # Bread staples
        "cat_Bot_Mar_Boter",             # Butter (basic)
    ]
    necessity_cols = [c for c in necessity_cols if c in df.columns]
    
    premium_spend = df[premium_cols].sum(axis=1) if premium_cols else 0
    necessity_spend = df[necessity_cols].sum(axis=1) if necessity_cols else 0
    
    # Premium ratio: lifestyle orientation vs survival shopping
    df["premium_ratio"] = np.where(
        (premium_spend + necessity_spend) > 0,
        premium_spend / (premium_spend + necessity_spend),
        0.5  # Neutral if no signal
    )
    
    # =========================================================================
    # STEP 7: CLEAN OTHER NUMERIC FEATURES
    # =========================================================================
    df["rev_ticket"] = pd.to_numeric(df.get("rev_ticket", 0), errors="coerce").fillna(0)
    df["prod_ticket"] = pd.to_numeric(df.get("prod_ticket", 0), errors="coerce").fillna(0)
    df["price_sens_colr"] = pd.to_numeric(df.get("price_sens_colr", 0), errors="coerce").fillna(0)
    df["SOW_colr"] = pd.to_numeric(df.get("SOW_colr", 0), errors="coerce").fillna(0)
    
    # Keep the wine anchor feature directly
    df["cat_Wijn_Stillewijnen_RAYON"] = pd.to_numeric(
        df.get("cat_Wijn_Stillewijnen_RAYON", 0), errors="coerce"
    ).fillna(0)
    
    # =========================================================================
    # STEP 8: HANDLE CATEGORICAL FEATURES
    # =========================================================================
    # HOUSEHOLDTYPOLOGY: normalize "!" to "unknown"
    if "HOUSEHOLDTYPOLOGY" in df.columns:
        df["HOUSEHOLDTYPOLOGY"] = (
            df["HOUSEHOLDTYPOLOGY"]
            .fillna("unknown")
            .replace("!", "unknown")
            .astype(str)
        )
    
    # SOW_type_colr: handle outlier flags
    if "SOW_type_colr" in df.columns:
        df["SOW_type_colr"] = (
            df["SOW_type_colr"]
            .fillna("unknown")
            .replace("!", "unknown")
            .astype(str)
        )
    
    # =========================================================================
    # STEP 9: SELECT FINAL FEATURE SET
    # =========================================================================
    final_numeric = [
        "total_spend",
        "rev_ticket",
        "prod_ticket",
        "n_cogo",
        "cogo_rev",
        "online_ratio",
        "price_sens_colr",
        "discount_ratio",
        "cat_Wijn_Stillewijnen_RAYON",
        "wine_affinity_spend",
        "wine_affinity_ratio",
        "premium_ratio",
        "SOW_colr",
    ]
    
    final_categorical = [
        "HOUSEHOLDTYPOLOGY",
        "SOW_type_colr",
    ]
    
    # Only keep columns that exist
    final_numeric = [c for c in final_numeric if c in df.columns]
    final_categorical = [c for c in final_categorical if c in df.columns]
    
    # Return only the curated features
    return df[final_numeric + final_categorical], final_numeric, final_categorical

In [44]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # numeric coercion (NO filling here)
    num_cols = [c for c in df.columns if c.startswith("cat_")] + [
        "rev_ticket", "prod_ticket", "n_cogo", "cogo_rev",
        "total_discount", "price_sens_colr", "SOW_colr"
    ]
    num_cols = [c for c in num_cols if c in df.columns]

    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # SOW normalization + flag
    if "SOW_type_colr" in df.columns:
        df["SOW_!"] = (df["SOW_type_colr"] == "!").astype(int)
        df["SOW_type_colr"] = (
            df["SOW_type_colr"]
            .fillna("unknown")
            .replace("!", "unknown")
        )

    # Household normalization
    if "HOUSEHOLDTYPOLOGY" in df.columns:
        df["HOUSEHOLDTYPOLOGY"] = (
            df["HOUSEHOLDTYPOLOGY"]
            .fillna("unknown")
            .replace("!", "unknown")
        )

    # Negative value flags
    for c in [col for col in df.columns if col.startswith("cat_")]:
        negs = df[c] < 0
        df[f"{c}_neg_flag"] = negs.astype(int)

    return df


In [45]:
from sklearn.base import BaseEstimator, TransformerMixin

class ClipLog(BaseEstimator, TransformerMixin):
    """Log1p transform with negative clipping. Supports get_feature_names_out."""
    
    def fit(self, X, y=None):
        # Store feature names if available
        if hasattr(X, 'columns'):
            self._feature_names = list(X.columns)
        elif hasattr(X, 'shape'):
            self._feature_names = [f"x{i}" for i in range(X.shape[1])]
        return self

    def transform(self, X):
        arr = np.array(X, dtype=float, copy=True)
        arr[arr < 0] = 0
        return np.log1p(arr)
    
    def get_feature_names_out(self, input_features=None):
        """Required for sklearn pipeline feature name propagation."""
        if input_features is not None:
            return np.array(input_features)
        return np.array(self._feature_names)

In [46]:
# #making categories 
# # Spend / turnover-like features
# spend_cols = [col for col in X_train.columns if col.startswith("cat_")] + [
#     "cogo_rev",
#     "total_discount",
#     "rev_ticket"
# ]

# # Count-like features (can also be scaled)
# count_cols = [
#     "prod_ticket",
#     "n_cogo"
# ]

# # Other numeric features
# other_numeric_cols = [
#     "price_sens_colr",
#     "SOW_colr"
# ]

# numeric_cols = spend_cols + count_cols + other_numeric_cols

# categorical_cols = [
#     "HOUSEHOLDTYPOLOGY",
#     "SOW_type_colr"
# ]

import numpy as np
import pandas as pd

def infer_feature_groups(df: pd.DataFrame,
                          skew_thresh=1.0,
                          range_ratio_thresh=20):
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

    # ---- Binary (0/1) ----
    binary_cols = []
    for c in numeric_cols:
        vals = df[c].dropna().unique()
        if len(vals) <= 2 and set(vals).issubset({0, 1}):
            binary_cols.append(c)

    # ---- Bounded [0,1] but not binary we want to keep ----
    bounded_cols = []
    for c in numeric_cols:
        if c in binary_cols:
            continue
        col = df[c].dropna()
        if len(col) and col.min() >= 0 and col.max() <= 1:
            bounded_cols.append(c)

    log_cols = [
        c for c in numeric_cols
        if c not in binary_cols
        and c not in bounded_cols
        
    ]

    return {
        "log_numeric": log_cols,
        "binary": binary_cols,
        "categorical": categorical_cols,
    }


In [47]:
def finalpipeline(X_train: pd.DataFrame):
    """
    Senior Data Scientist Pipeline
    
    Key design choices:
    - ElasticNet regularization (handles correlated features properly)
    - Log-transform for skewed spend features
    - StandardScaler for all numeric (critical for regularization)
    - OneHotEncoder for categoricals
    
    ElasticNet rationale:
    - L2 component stabilizes correlated features
    - L1 component performs feature selection if redundant
    - Best of both worlds for overlapping features
    """
    # Apply feature engineering first
    X_engineered, numeric_cols, categorical_cols = final_features(X_train)
    
    # Separate log-transform candidates (spend/revenue features) from others
    log_transform_cols = [
        "total_spend",
        "cogo_rev",
        "cat_Wijn_Stillewijnen_RAYON",
        "wine_affinity_spend",
        "rev_ticket",
    ]
    log_transform_cols = [c for c in log_transform_cols if c in numeric_cols]
    
    # Non-log numeric (ratios, counts, scores - already bounded or normalized)
    standard_numeric_cols = [c for c in numeric_cols if c not in log_transform_cols]
    
    # Log-transform pipeline (for skewed spend features)
    log_numeric_pipeline = Pipeline([
        ("impute", SimpleImputer(strategy="constant", fill_value=0)),
        ("clip_log", ClipLog()),
        ("scaler", StandardScaler())
    ])
    
    # Standard numeric pipeline (for ratios, scores, counts)
    standard_numeric_pipeline = Pipeline([
        ("impute", SimpleImputer(strategy="constant", fill_value=0)),
        ("scaler", StandardScaler())
    ])
    
    # Categorical pipeline
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
    
    categorical_pipeline = Pipeline([
        ("impute", SimpleImputer(strategy="constant", fill_value="missing")),
        ("ohe", ohe)
    ])
    
    # Column transformer with curated features
    preprocessor = ColumnTransformer(
        transformers=[
            ("log_num", log_numeric_pipeline, log_transform_cols),
            ("std_num", standard_numeric_pipeline, standard_numeric_cols),
            ("cat", categorical_pipeline, categorical_cols),
        ],
        remainder="drop",
        verbose_feature_names_out=False
    )
    
    # Full model pipeline with ElasticNet
    # Senior choice: ElasticNet handles overlapping features properly
    model = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("classifier", LogisticRegression(
            penalty="elasticnet",
            solver="saga",          # Only solver that supports elasticnet
            l1_ratio=0.5,           # Balance between L1 and L2
            C=1.0,                  # Regularization strength (will tune)
            max_iter=5000,          # saga needs more iterations
            class_weight="balanced",
            random_state=42
        ))
    ])
    
    feature_groups = {
        "log_numeric": log_transform_cols,
        "standard_numeric": standard_numeric_cols,
        "categorical": categorical_cols,
    }
    
    return model, feature_groups, X_engineered

In [48]:
# Apply feature engineering to train and test sets
X_train_eng, num_cols, cat_cols = final_features(X_train)
X_test_eng, _, _ = final_features(X_test)

print(f"Engineered features: {X_train_eng.shape[1]} total")
print(f"  Numeric: {len(num_cols)}")
print(f"  Categorical: {len(cat_cols)}")
print(f"\nNumeric features: {num_cols}")
print(f"Categorical features: {cat_cols}")

# Build the pipeline with ElasticNet
gold_model, feature_groups, _ = finalpipeline(X_train)

# Show feature groups
print("\n" + "="*60)
print("FEATURE GROUPS")
print("="*60)
for k, v in feature_groups.items():
    print(f"\n{k.upper()} ({len(v)}):")
    for c in v:
        print("  ", c)

Engineered features: 15 total
  Numeric: 13
  Categorical: 2

Numeric features: ['total_spend', 'rev_ticket', 'prod_ticket', 'n_cogo', 'cogo_rev', 'online_ratio', 'price_sens_colr', 'discount_ratio', 'cat_Wijn_Stillewijnen_RAYON', 'wine_affinity_spend', 'wine_affinity_ratio', 'premium_ratio', 'SOW_colr']
Categorical features: ['HOUSEHOLDTYPOLOGY', 'SOW_type_colr']

FEATURE GROUPS

LOG_NUMERIC (5):
   total_spend
   cogo_rev
   cat_Wijn_Stillewijnen_RAYON
   wine_affinity_spend
   rev_ticket

STANDARD_NUMERIC (8):
   prod_ticket
   n_cogo
   online_ratio
   price_sens_colr
   discount_ratio
   wine_affinity_ratio
   premium_ratio
   SOW_colr

CATEGORICAL (2):
   HOUSEHOLDTYPOLOGY
   SOW_type_colr


In [49]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import make_scorer, average_precision_score, precision_recall_curve
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# STEP 1: STABILITY CHECK (Senior approach - fixed params first)
# =============================================================================
print("="*60)
print("STEP 1: STABILITY CHECK (Fixed Parameters)")
print("="*60)
print("Running CV with fixed params to check variance before tuning...")

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Use average_precision (better for imbalanced data than F1)
stability_scores = cross_validate(
    gold_model,
    X_train_eng,
    y_train,
    cv=cv,
    scoring={
        'average_precision': 'average_precision',
        'f1': 'f1',
        'precision': 'precision',
        'recall': 'recall'
    },
    return_train_score=True
)

print(f"\nAverage Precision: {stability_scores['test_average_precision'].mean():.4f} ± {stability_scores['test_average_precision'].std():.4f}")
print(f"F1 Score:          {stability_scores['test_f1'].mean():.4f} ± {stability_scores['test_f1'].std():.4f}")
print(f"Precision:         {stability_scores['test_precision'].mean():.4f} ± {stability_scores['test_precision'].std():.4f}")
print(f"Recall:            {stability_scores['test_recall'].mean():.4f} ± {stability_scores['test_recall'].std():.4f}")

# Check if variance is acceptable (std < 0.05 is good)
ap_std = stability_scores['test_average_precision'].std()
if ap_std < 0.03:
    print("\n✅ Model is VERY STABLE (std < 0.03)")
elif ap_std < 0.05:
    print("\n✅ Model is STABLE (std < 0.05)")
else:
    print(f"\n⚠️ Model has HIGH VARIANCE (std = {ap_std:.4f}) - proceed with caution")

STEP 1: STABILITY CHECK (Fixed Parameters)
Running CV with fixed params to check variance before tuning...

Average Precision: 0.4098 ± 0.0096
F1 Score:          0.3123 ± 0.0019
Precision:         0.1924 ± 0.0011
Recall:            0.8295 ± 0.0099

✅ Model is VERY STABLE (std < 0.03)


In [31]:
# =============================================================================
# STEP 2: COEFFICIENT STABILITY ANALYSIS
# =============================================================================
print("="*60)
print("STEP 2: COEFFICIENT STABILITY ACROSS FOLDS")
print("="*60)

from sklearn.base import clone

def get_feature_names(model, feature_groups):
    """Simple helper to get feature names from our pipeline."""
    names = feature_groups['log_numeric'] + feature_groups['standard_numeric']
    # Add OHE categories
    ohe = model.named_steps['preprocess'].named_transformers_['cat'].named_steps['ohe']
    for col_idx, col in enumerate(feature_groups['categorical']):
        for cat in ohe.categories_[col_idx]:
            names.append(f"{col}_{cat}")
    return names

# Collect coefficients from each fold
fold_coefs = []
fold_feature_names = None

for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train_eng, y_train)):
    X_fold_train = X_train_eng.iloc[train_idx]
    y_fold_train = y_train.iloc[train_idx]
    
    fold_model = clone(gold_model)
    fold_model.fit(X_fold_train, y_fold_train)
    fold_coefs.append(fold_model.named_steps['classifier'].coef_[0])
    
    if fold_feature_names is None:
        fold_feature_names = get_feature_names(fold_model, feature_groups)

fold_coefs = np.array(fold_coefs)

# Check sign stability
sign_stability = (np.sign(fold_coefs) == np.sign(fold_coefs.mean(axis=0))).mean(axis=0)
unstable_mask = sign_stability < 0.8

print(f"\nTotal features: {len(fold_feature_names)}")
print(f"Stable features (same sign ≥80% folds): {(~unstable_mask).sum()}")
print(f"Unstable features (sign flips): {unstable_mask.sum()}")

if unstable_mask.any():
    print("\n⚠️ Unstable features:")
    for name, stab in zip(fold_feature_names, sign_stability):
        if stab < 0.8:
            print(f"   {name}: stable in {stab*100:.0f}% of folds")
else:
    print("\n✅ All features have stable coefficient signs!")

# Top features
print("\n" + "-"*60)
print("TOP 10 FEATURES BY MEAN |COEFFICIENT|")
print("-"*60)
mean_coefs = fold_coefs.mean(axis=0)
std_coefs = fold_coefs.std(axis=0)
top_idx = np.argsort(np.abs(mean_coefs))[-10:][::-1]

for idx in top_idx:
    sign = "+" if mean_coefs[idx] > 0 else "-"
    print(f"  {sign} {fold_feature_names[idx]:40s} coef={mean_coefs[idx]:+.3f} ± {std_coefs[idx]:.3f}")

STEP 2: COEFFICIENT STABILITY ACROSS FOLDS

Total features: 39
Stable features (same sign in ≥80% folds): 36
Unstable features (sign flips): 3

⚠️ Unstable features (coefficients flip sign):
   HOUSEHOLDTYPOLOGY_f_HHnochild_35_54: stable in 60% of folds
   HOUSEHOLDTYPOLOGY_k_HHchild_oldest_13_17: stable in 40% of folds
   SOW_type_colr_unknown: stable in 60% of folds

------------------------------------------------------------
TOP 10 FEATURES BY MEAN |COEFFICIENT|
------------------------------------------------------------
  + rev_ticket                               coef=+1.980 ± 0.019 (stable: 100%)
  + wine_affinity_spend                      coef=+1.755 ± 0.046 (stable: 100%)
  - SOW_type_colr_SOW00-10                   coef=-1.374 ± 0.043 (stable: 100%)
  + SOW_type_colr_Outlier_fr                 coef=+1.331 ± 0.081 (stable: 100%)
  - discount_ratio                           coef=-1.097 ± 0.096 (stable: 100%)
  - SOW_type_colr_SOW10-20                   coef=-1.087 ± 0.035 (st

In [24]:
# =============================================================================
# STEP 3: SMALL HYPOTHESIS-DRIVEN GRID SEARCH
# =============================================================================
print("="*60)
print("STEP 3: HYPOTHESIS-DRIVEN TUNING")
print("="*60)
print("Senior approach: Small grid, hypothesis-driven, not slot machine")

from sklearn.model_selection import GridSearchCV

# Small, focused grid (not 200 combinations)
# Hypothesis: ElasticNet with moderate regularization should work best
param_grid = {
    'classifier__C': [0.1, 0.5, 1.0, 2.0],      # Regularization strength
    'classifier__l1_ratio': [0.3, 0.5, 0.7],    # L1 vs L2 balance
}

print(f"\nGrid size: {len(param_grid['classifier__C']) * len(param_grid['classifier__l1_ratio'])} combinations")
print("Scoring: average_precision (better for imbalanced marketing problems)")

grid_search = GridSearchCV(
    estimator=gold_model,
    param_grid=param_grid,
    scoring='average_precision',  # Senior choice for imbalanced data
    cv=cv,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

grid_search.fit(X_train_eng, y_train)

print(f"\n✅ Best Average Precision: {grid_search.best_score_:.4f}")
print(f"   Best C: {grid_search.best_params_['classifier__C']}")
print(f"   Best l1_ratio: {grid_search.best_params_['classifier__l1_ratio']}")

# Compare default vs tuned
baseline_ap = stability_scores['test_average_precision'].mean()
tuned_ap = grid_search.best_score_
improvement = (tuned_ap - baseline_ap) / baseline_ap * 100

print(f"\n📊 Improvement from tuning: {improvement:+.2f}%")
if improvement < 2:
    print("   → Minimal improvement. Default params are fine.")
elif improvement < 5:
    print("   → Modest improvement. Use tuned params.")
else:
    print("   → Significant improvement. Tuning was worthwhile.")

STEP 3: HYPOTHESIS-DRIVEN TUNING
Senior approach: Small grid, hypothesis-driven, not slot machine

Grid size: 12 combinations
Scoring: average_precision (better for imbalanced marketing problems)
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/pranavreddy/Desktop/projects/wines/Highbrow-wines-case-predictive-modelling-/.venv-py39/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/pranavreddy/Desktop/projects/wines/Highbrow-wines-case-predictive-modelling-/.venv-py39/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/pranavreddy/Desktop/projects/wines/Highbrow-wines-case-predictive-modelling-/.venv-py39/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/pranavreddy/Desktop/projects/wines/Highbrow-wines-case-predictive-modelling-/.venv-py39/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ di


✅ Best Average Precision: 0.4098
   Best C: 2.0
   Best l1_ratio: 0.5

📊 Improvement from tuning: +0.01%
   → Minimal improvement. Default params are fine.


In [25]:
# =============================================================================
# STEP 4: THRESHOLD TUNING (Senior approach: tune threshold, not just weights)
# =============================================================================
from sklearn.metrics import precision_recall_curve, classification_report, confusion_matrix

print("="*60)
print("STEP 4: THRESHOLD TUNING")
print("="*60)
print("Senior insight: Tune the decision threshold, not just model weights")

# Get best model and predict probabilities
best_model = grid_search.best_estimator_
y_proba = best_model.predict_proba(X_test_eng)[:, 1]

# Calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

# Find optimal threshold for different business objectives
print("\n📊 Threshold Analysis:")
print("-"*60)

# Objective 1: Maximize F1
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
best_f1_idx = np.argmax(f1_scores[:-1])  # exclude last element
best_f1_threshold = thresholds[best_f1_idx]
print(f"Max F1 threshold:           {best_f1_threshold:.3f} (F1={f1_scores[best_f1_idx]:.3f})")

# Objective 2: Recall @ Precision >= 0.7 (realistic for marketing)
mask_p70 = precision[:-1] >= 0.7
if mask_p70.any():
    best_recall_at_p70 = recall[:-1][mask_p70].max()
    idx_p70 = np.where((recall[:-1] == best_recall_at_p70) & mask_p70)[0][0]
    threshold_p70 = thresholds[idx_p70]
    print(f"Max Recall @ Precision≥70%: {threshold_p70:.3f} (Recall={best_recall_at_p70:.3f})")
else:
    print("Max Recall @ Precision≥70%: Not achievable")
#objective 3: Recall @ Precision >= 0.5 (balanced)
mask_p50 = precision[:-1] >= 0.5
if mask_p50.any():
    best_recall_at_p50 = recall[:-1][mask_p50].max()
    idx_p50 = np.where((recall[:-1] == best_recall_at_p50) & mask_p50)[0][0]
    threshold_p50 = thresholds[idx_p50]
    print(f"Max Recall @ Precision≥50%: {threshold_p50:.3f} (Recall={best_recall_at_p50:.3f})")
else:
    print("Max Recall @ Precision≥50%: Not achievable")

# Objective 4: Recall @ Precision >= 0.3 (broader reach)
mask_p30 = precision[:-1] >= 0.3
if mask_p30.any():
    best_recall_at_p30 = recall[:-1][mask_p30].max()
    idx_p30 = np.where((recall[:-1] == best_recall_at_p30) & mask_p30)[0][0]
    threshold_p30 = thresholds[idx_p30]
    print(f"Max Recall @ Precision≥30%: {threshold_p30:.3f} (Recall={best_recall_at_p30:.3f})")
else:
    print("Max Recall @ Precision≥30%: Not achievable")

# Use best F1 threshold for final evaluation
CHOSEN_THRESHOLD = best_f1_threshold
print(f"\n🎯 Using threshold: {CHOSEN_THRESHOLD:.3f} (Max F1 objective)")

STEP 4: THRESHOLD TUNING
Senior insight: Tune the decision threshold, not just model weights

📊 Threshold Analysis:
------------------------------------------------------------
Max F1 threshold:           0.834 (F1=0.441)
Max Recall @ Precision≥70%: 0.986 (Recall=0.127)
Max Recall @ Precision≥50%: 0.914 (Recall=0.328)
Max Recall @ Precision≥30%: 0.721 (Recall=0.630)

🎯 Using threshold: 0.834 (Max F1 objective)


In [26]:
# =============================================================================
# STEP 5: FINAL TEST SET EVALUATION
# =============================================================================
print("="*60)
print("STEP 5: FINAL TEST SET EVALUATION")
print("="*60)

# Predictions with tuned threshold
y_pred_tuned = (y_proba >= CHOSEN_THRESHOLD).astype(int)
y_pred_default = best_model.predict(X_test_eng)  # default 0.5 threshold

print("\n📊 Classification Report (Tuned Threshold):")
print(classification_report(y_test, y_pred_tuned, target_names=["No Highbrow", "Highbrow"]))

print("\n📊 Confusion Matrix (Tuned Threshold):")
cm = confusion_matrix(y_test, y_pred_tuned)
tn, fp, fn, tp = cm.ravel()
print(f"                    Predicted")
print(f"                No         Yes")
print(f"Actual No    {tn:5d}       {fp:5d}")
print(f"Actual Yes   {fn:5d}       {tp:5d}")

print("\n📊 Business Metrics:")
print(f"   True Positives (correctly identified buyers): {tp}")
print(f"   False Positives (wasted marketing): {fp}")
print(f"   False Negatives (missed buyers): {fn}")
print(f"   True Negatives (correctly ignored): {tn}")

# Marketing efficiency
if tp + fp > 0:
    precision_val = tp / (tp + fp)
    print(f"\n   📧 If you target {tp + fp} customers:")
    print(f"      → {tp} will buy ({precision_val*100:.1f}% hit rate)")
    print(f"      → {fp} won't buy (wasted effort)")

# Compare default vs tuned threshold
print("\n" + "-"*60)
print("📊 THRESHOLD COMPARISON:")
print("-"*60)

cm_default = confusion_matrix(y_test, y_pred_default)
tn_d, fp_d, fn_d, tp_d = cm_default.ravel()

print(f"Default (0.50): Precision={tp_d/(tp_d+fp_d):.3f}, Recall={tp_d/(tp_d+fn_d):.3f}, TP={tp_d}, FP={fp_d}")
print(f"Tuned ({CHOSEN_THRESHOLD:.2f}):   Precision={precision_val:.3f}, Recall={tp/(tp+fn):.3f}, TP={tp}, FP={fp}")

STEP 5: FINAL TEST SET EVALUATION

📊 Classification Report (Tuned Threshold):
              precision    recall  f1-score   support

 No Highbrow       0.97      0.96      0.97     38008
    Highbrow       0.40      0.49      0.44      1992

    accuracy                           0.94     40000
   macro avg       0.69      0.73      0.70     40000
weighted avg       0.94      0.94      0.94     40000


📊 Confusion Matrix (Tuned Threshold):
                    Predicted
                No         Yes
Actual No    36550        1458
Actual Yes    1017         975

📊 Business Metrics:
   True Positives (correctly identified buyers): 975
   False Positives (wasted marketing): 1458
   False Negatives (missed buyers): 1017
   True Negatives (correctly ignored): 36550

   📧 If you target 2433 customers:
      → 975 will buy (40.1% hit rate)
      → 1458 won't buy (wasted effort)

------------------------------------------------------------
📊 THRESHOLD COMPARISON:
------------------------------

In [30]:
# =============================================================================
# STEP 6: FEATURE IMPORTANCE (Senior interpretation)
# =============================================================================
print("="*60)
print("STEP 6: FEATURE IMPORTANCE INTERPRETATION")
print("="*60)
print("Senior rule: Interpret coefficients as 'holding X constant...'")

# Get feature names and coefficients
feature_names = get_feature_names(best_model, feature_groups)
coefs = best_model.named_steps['classifier'].coef_[0]

print(f"\nTotal features: {len(coefs)}")

# Create sorted DataFrame
coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefs,
}).assign(abs_coef=lambda x: np.abs(x.coefficient)).sort_values('abs_coef', ascending=False)

print("\n🔝 TOP 15 FEATURES BY IMPORTANCE:")
print("-"*60)
for _, row in coef_df.head(15).iterrows():
    direction = "↑" if row['coefficient'] > 0 else "↓"
    print(f"  {direction} {row['feature']:40s} {row['coefficient']:+.4f}")

print("\n📖 INTERPRETATION:")
print("• ↑ Positive = INCREASES likelihood of buying highbrow wine")
print("• ↓ Negative = DECREASES likelihood of buying highbrow wine")

print("\n✅ KEY POSITIVE DRIVERS:")
for f in coef_df[coef_df['coefficient'] > 0].head(5)['feature']:
    print(f"   • {f}")

print("\n❌ KEY NEGATIVE DRIVERS:")
for f in coef_df[coef_df['coefficient'] < 0].head(5)['feature']:
    print(f"   • {f}")

STEP 6: FEATURE IMPORTANCE INTERPRETATION
Senior rule: Interpret coefficients as 'holding X constant...'

Total features: 39
Feature names extracted: 39

🔝 TOP 15 FEATURES BY IMPORTANCE:
------------------------------------------------------------
  ↑ rev_ticket                                    +1.9810
  ↑ wine_affinity_spend                           +1.7559
  ↓ SOW_type_colr_SOW00-10                        -1.4012
  ↑ SOW_type_colr_Outlier_fr                      +1.3475
  ↓ SOW_type_colr_SOW10-20                        -1.1087
  ↓ discount_ratio                                -1.0940
  ↓ total_spend                                   -1.0377
  ↑ SOW_type_colr_Outlier_om                      +0.8425
  ↓ prod_ticket                                   -0.8292
  ↓ SOW_type_colr_SOW20-30                        -0.8053
  ↓ wine_affinity_ratio                           -0.6657
  ↓ HOUSEHOLDTYPOLOGY_i_HHchild_oldest_0_5        -0.5614
  ↓ HOUSEHOLDTYPOLOGY_a_Single_18_34              -0.538

Fitting 5 folds for each of 16 candidates, totalling 80 fits



KeyboardInterrupt



1.4.2
